In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_gigachat.chat_models import GigaChat
from langchain_core.prompts import PromptTemplate


giga = GigaChat(
    # Для авторизации запросов используйте ключ, полученный в проекте GigaChat API
    credentials="YjBiMGE1YWQtMjNiMS00YTI0LTg4MGItN2NmYjYzNTE5M2RhOmRmNjM1ZGRjLThmYTEtNDViNy1iMDA0LTFmODZiMGY4NTgxNA==",
    verify_ssl_certs=False,
)


def read_email_from_file(file_path: str):
    """Считывает тему, отправителя и текст письма из файла."""
    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()
    sender = lines[0].strip()
    subject = lines[1].strip()
    content = " ".join(line.strip() for line in lines[2:])
    return subject, sender, content

# Функция для анализа важности письма
def analyze_email(subject: str, sender: str, content: str) -> str:
    """Анализирует письмо и определяет его важность."""
    prompt_template = PromptTemplate(
        input_variables=["subject", "sender", "content"],
        template=(
            "Тема письма: {subject}\n"
            "Отправитель: {sender}\n"
            "Текст письма: {content}\n\n"
            "Определи, насколько это письмо важно для получателя (высокая, средняя, низкая важность) и объясни почему."
        ),
    )
    
    prompt = prompt_template.format(subject=subject, sender=sender, content=content)
    response = giga([HumanMessage(content=prompt)])
    
    return response.content

# Пример использования
if __name__ == "__main__":
    file_path = "email_test_v1/email3.txt"  # Укажите путь к файлу с письмом
    subject, sender, content = read_email_from_file(file_path)
    importance = analyze_email(subject, sender, content)
    print("Результат анализа:", importance)

Тест-кейс 1
trigger = false
boss = false

Тест-кейс 2
trigger = false
boss = true

Тест-кейс 3
trigger = true
boss = false

Тест-кейс 4
trigger = true
boss = true